In [8]:
import csv
import json
import random
from datetime import datetime

# ============================
# CONFIG
# ============================

WORKS_CSV = "works.csv"  # must be in the same folder

USER_LIST = [
    {"id": 1, "name": "Kanye West", "password": "Graduation"},
    {"id": 2, "name": "Stanley Kubrick", "password": "2001: A Space Odyssey"},
    {"id": 3, "name": "Ludwig van Beethoven", "password": "Symphony no.9"},
    {"id": 4, "name": "Quentin Tarantino", "password": "Pulp Fiction"},
    {"id": 5, "name": "Kendrick Lamar", "password": "Good Kid, M.A.A.D City"},
    {"id": 6, "name": "Ernest Hemingway", "password": "A Farewell to Arms"},
    {"id": 7, "name": "Leonardo DiCaprio", "password": "The Wolf of Wall Street"},
    {"id": 8, "name": "Frank Lloyd Wright", "password": "Fallingwater"},
    {"id": 9, "name": "Vincent van Gogh", "password": "Starry Night"},
    {"id": 10, "name": "Steve Jobs", "password": "iPod"},
    {"id": 11, "name": "Elon Musk", "password": "SpaceX"},
    {"id": 12, "name": "Albert Einstein", "password": "Special Relativity"},
    {"id": 13, "name": "Isaac Newton", "password": "Calculus"},
    {"id": 14, "name": "F. Scott Fitzgerald", "password": "The Great Gatsby"},
    {"id": 15, "name": "David Lynch", "password": "Twin Peaks"},
    {"id": 16, "name": "Denis Villeneuve", "password": "Blade Runner 2049"},
    {"id": 17, "name": "The Beatles", "password": "Sgt. Pepper"},
]

ACCOUNT_TYPES = ["Checking", "Savings", "Credit"]


# ============================
# HELPERS
# ============================

def today_yyyymmdd():
    return int(datetime.now().strftime("%Y%m%d"))


def create_accounts(base_number):
    """Create 3 bank accounts starting with $1000"""
    accounts = []
    for i, acc_type in enumerate(ACCOUNT_TYPES):
        accounts.append({
            "number": str(base_number + i),
            "type": acc_type,
            "balance": 1000.0,
            "transactions": []
        })
    return accounts


def pick_random_account(user_json):
    return random.choice(user_json["accounts"])


# ============================
# LOAD WORKS CSV
# ============================

works = []
with open(WORKS_CSV, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        works.append({
            "creator": row["Creator"],
            "work": row["Work"],
            "date": row["Date"],
            "type": row["Type"],
            "price": float(row["Price"]),
        })


# ============================
# BUILD INITIAL USERS
# ============================

users = []
next_acc_base = 1001

for u in USER_LIST:
    users.append({
        "id": u["id"],
        "name": u["name"],
        "role": "customer",
        "password": u["password"],
        "accounts": create_accounts(next_acc_base)
    })
    next_acc_base += 10


# ============================
# SIMULATE TRANSACTIONS
# ============================

for buyer in users:

    # All works EXCEPT those by the buyer
    valid_works = [w for w in works if w["creator"] != buyer["name"]]

    # Always choose 20–100 works from ANY creator (with replacement)
    k = random.randint(20, 100)
    chosen = [random.choice(valid_works) for _ in range(k)]
    for w in chosen:
        price = w["price"]

        buyer_acc = pick_random_account(buyer)

        # Find seller
        seller = next((x for x in users if x["name"] == w["creator"]), None)
        if seller is None:
            continue

        seller_acc = pick_random_account(seller)

        buyer_acc["balance"] -= price  # allow negative
        seller_acc["balance"] += price

        # Buyer → transaction
        buyer_acc["transactions"].append({
            "date": today_yyyymmdd(),
            "amount": -price,
            "details": f"Purchased '{w['work']}' from {w['creator']}",
            "to_account": seller_acc["number"],
            "from_account": buyer_acc["number"],
            "recipient_id": seller["id"],
            "recipient_name": seller["name"]
        })

        # Seller → transaction
        seller_acc["transactions"].append({
            "date": today_yyyymmdd(),
            "amount": price,
            "details": f"Sold '{w['work']}' to {buyer['name']}",
            "to_account": seller_acc["number"],
            "from_account": buyer_acc["number"],
            "recipient_id": buyer["id"],
            "recipient_name": buyer["name"]
        })


# ============================
# OUTPUT FINAL JSON
# ============================

with open("bank_users.json", "w", encoding="utf-8") as f:
    json.dump(users, f, indent=2)

print("DONE → bank_users.json created.")

DONE → bank_users.json created.
